### Data Ingestion with LangChain4j
<img src="https://docs.langchain4j.dev/img/logo.svg" alt="LangChain4J" width="200" height="200">

In [9]:
%use dataframe
%useLatestDescriptors

import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import dev.langchain4j.data.document.*
import dev.langchain4j.data.segment.TextSegment
import dev.langchain4j.store.embedding.EmbeddingStore
import dev.langchain4j.store.embedding.pgvector.PgVectorEmbeddingStore
import java.util.Arrays

### Create Menu

In [10]:
import org.jetbrains.kotlinx.dataframe.codeGen.generateCode
val df = DataFrame.read("src/main/resources/food/italian_delaight_dishes.csv")
df.generateCode()
df.head()


Name,Category,Ingredients
Tiramisu,Desserts,"[['Mascarpone', '750g'], ['Eggs', '26..."
Cookies,Desserts,"[['Flour', '195g'], ['Butter', '100g'..."
Pancakes with Maple Syrup,Desserts,"[['Butter', '25g'], ['Flour', '125g']..."
Mascarpone Cream,Desserts,"[['Mascarpone', '500g'], ['Sugar', '1..."
Sweet and Savory Crepes (Basic Recipe),Desserts,"[['Eggs', '3'], ['Flour', '250g'], ['..."


In [11]:

val mapper = jacksonObjectMapper()
val categories = listOf("Desserts", "Main courses", "Sides", "Appetizers", "Second Courses", "First Courses")
val markdown = df.groupBy("Category")
    .filter{categories.contains(Category)}
    .map { group ->
        val category = group.key["Category"]
        val dishes = group.group.map { row ->
            runCatching {
            "- Meal: ${row.Name}\n\t- Ingredients: ${mapper.readValue<List<List<String>>>(row.Ingredients.replace("'", "\"")).map { it[0] }[0]}\n"}.getOrNull() ?: ""
        }.joinToString("")

        "### Category: $category\n$dishes"
    }.filterNotNull().joinToString("\n\n")

markdown


### Category: Desserts
- Meal: Tiramisu
	- Ingredients: Mascarpone
- Meal: Cookies
	- Ingredients: Flour
- Meal: Pancakes with Maple Syrup
	- Ingredients: Butter
- Meal: Mascarpone Cream
	- Ingredients: Mascarpone
- Meal: Sweet and Savory Crepes (Basic Recipe)
	- Ingredients: Eggs
- Meal: Gingerbread Cookies
	- Ingredients: Ground ginger
- Meal: New York Cheesecake
	- Ingredients: Digestive biscuits
- Meal: Molten Chocolate Cake
	- Ingredients: 70% dark chocolate
- Meal: Pastry Cream
	- Ingredients: Whole milk
- Meal: Nutella Crepes
	- Ingredients: Eggs
- Meal: Chocolate Chip Muffins
	- Ingredients: Butter
- Meal: Sicilian Cannoli
	- Ingredients: Flour
- Meal: Tenerina Cake
	- Ingredients: Dark chocolate
- Meal: Brownies
	- Ingredients: Dark chocolate
- Meal: Wine Cookies
	- Ingredients: White wine
- Meal: Apple Cake
	- Ingredients: Renette apples
- Meal: French Toast
	- Ingredients: Eggs
- Meal: Fluffy Brioche Leavened Cake
	- Ingredients: Manitoba flour
- Meal: Shortcrust Pastry
	- I